In [1]:
from datasets import Dataset
import os

from watchfiles import awatch

base_path = "/home/jfeil/MasterThesis/dataset_distillation/base-v5-unshuffled/"
output_path = "/home/jfeil/MasterThesis/dataset_distillation/v1/"

if not os.path.exists(output_path):
    os.makedirs(output_path)

def load_dataset(path):
    dataset = Dataset.from_parquet(path)
    dataset = dataset.rename_column("gt", "wiktionary_gt")
    return dataset

train_set = load_dataset(base_path + "train.parquet")
test_set = load_dataset(base_path + "test.parquet")
val_set = load_dataset(base_path + "val.parquet")

In [2]:
train_set[24000]

{'title': 'Verhältnis',
 'context_word': 'Verhältnis',
 'context_sentence': '"Evas Verhältnis ist nach Brüssel, da kann sie die Mietze zu sich einladen und ihr alles zeigen, wie es bei ganz feinen Leuten ist."',
 'wiktionary_gt': 'umgangssprachlich: intime Beziehung zwischen zwei Menschen',
 'prompt': '"Evas Verhältnis ist nach Brüssel, da kann sie die Mietze zu sich einladen und ihr alles zeigen, wie es bei ganz feinen Leuten ist." Was ist die Definition von Verhältnis? '}

In [25]:
from collections import Counter

x = Counter(train_set['wiktionary_gt'])

In [33]:
list(dict(x).items())[-1]

('allmähliche Verminderung des Tageslichts nach Sonnenuntergang; Zeitraum zwischen Sonnenuntergang und Eintritt der Dunkelheit',
 3)

In [21]:
len(train_set['wiktionary_gt'])

274875

In [22]:
1108/274875

0.004030923146884947

In [36]:
prefixes = [
    "salopp",
    "abwertend",
    "umgangssprachlich",
    "übertragen",
    "allgemein",
    "fachsprachlich",
    "familiär",
    "historisch",
    "gehoben",
    "landschaftlich",
    "norddeutsch", 
    "östereichisch",
    "scherzhaft",
    "schweizerisch",
    "süddeutsch",
    "figurativ",
    "veraltend"
]

prefixes_count = {p: 0 for p in prefixes}

count = 0
for gt in train_set['wiktionary_gt']:
    cur_gt = 0
    for prefix in prefixes:
        if prefix in gt:
            cur_gt += 1
            prefixes_count[prefix] += 1
    if cur_gt > 0:
        count += 1
count

26273

In [38]:
26273 / 274875

0.09558162801273307

In [37]:
prefixes_count

{'salopp': 764,
 'abwertend': 2839,
 'umgangssprachlich': 9533,
 'übertragen': 4655,
 'allgemein': 1878,
 'fachsprachlich': 732,
 'familiär': 144,
 'historisch': 2191,
 'gehoben': 1937,
 'landschaftlich': 552,
 'norddeutsch': 546,
 'östereichisch': 594,
 'scherzhaft': 982,
 'schweizerisch': 691,
 'süddeutsch': 503,
 'figurativ': 571,
 'veraltend': 1321}

In [3]:
def prompt_pattern(context: str, word: str, pattern="\"%s\": Was ist die Definition von %s?") -> str:
    return pattern % (context, word)


class Experiment:
    
    def __init__(self, system_prompt=None, question_prompt="\"%s\": Was ist die Definition von %s? ",
                 example_prompts=(
                         ("Die Liebe überwindet alle Grenzen", "Liebe",
                          "inniges Gefühl der Zuneigung für jemanden oder für etwas"))):
        if system_prompt:
            self.system_prompt = system_prompt
        else:
            self.system_prompt = ["Du bist ein Definitionsgenerator. "
                                  "Du antwortest in Deutsch. "
                                  "Du bekommst einen Beispielsatz und ein Wort aus dem Satz. "
                                  "Du antwortest mit der Definition des Wortes im Kontext des Beispiels. "
                                  "Du antwortest nur mit der Definition. "
                                  "Du zeigst den Bezug auf den Kontext. "
                                  "Du antwortest in maximal 5 Worten. "]
        self.question_prompt = question_prompt
        self.example_prompts = example_prompts

    def create_examples_prompt(self):
        ret_val = []
        for example, word, definition in self.example_prompts:
            ret_val += [prompt_pattern(example, word, pattern=self.question_prompt) + definition]
        return ret_val

    def create_examples(self):
        ret_val = []
        for example, word, definition in self.example_prompts:
            ret_val += [{
                "role": "user",
                "content": self.question_prompt % (example, word)
            },
                {
                    "role": "assistant",
                    "content": definition}
            ]
        return ret_val
    
default_experiment = Experiment(system_prompt=[
        "Du bist ein Definitionsgenerator. "
        "Du antwortest in Deutsch. "
        "Du bekommst einen Beispielsatz und ein Wort aus dem Satz. "
        "Du antwortest mit der Definition des Wortes im Kontext des Beispiels. "
        "Du antwortest nur mit der Definition. "
        "Du zeigst den Bezug auf den Kontext. "
        "Du antwortest so detailiert wie nötig. "
    ],
        question_prompt="%s: Was ist in diesem Kontext die Definition von %s?",
        example_prompts=[
            ("Die Liebe überwindet alle Grenzen",
             "Liebe",
             "inniges Gefühl der Zuneigung für jemanden oder für etwas"),
        ])

In [14]:
import csv

dataset = []

with open("mai_val_dataset.csv") as file:
    csv_reader = csv.reader(file, delimiter=',')
    next(csv_reader)
    for row in csv_reader:
        dataset.append(row)

In [18]:
dataset = dataset + dataset[0:32]

In [19]:
from tqdm.auto import tqdm
import os
import asyncio
from openai import AsyncOpenAI
import json
import datetime


base_url="http://localhost:8080/v1"

async_client = AsyncOpenAI(
    base_url=base_url,
    api_key="-"
)

def create_messages(prompt):
    return [
            {
                "role": "user",
                "content": prompt,
            }
        ]


async def async_prompt(row) -> None:
    return (await async_client.chat.completions.create(
        messages=create_messages(row[0]),
        model="tgi",
        stream=False,
        max_tokens=512, 
        frequency_penalty=1, 
        logprobs=False, 
        seed=42, 
        temperature=0.2,

    )).choices[0].message.content



async def prompt_dataset(dataset, split, experiment, batch_size=512, warm_up=32, checkpoint_dir="checkpoint"):
    prefix = str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
    
    if not os.path.isdir(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    
    responses = []
    tasks = []
    for i in tqdm(dataset):
        tasks.append(async_prompt(i))
    current_batch = await asyncio.gather(*tasks[0:warm_up])
    responses.append(current_batch)
    with open(f"{checkpoint_dir}/{split}_{prefix}_0.json", "w") as f:
        print(current_batch)
        json.dump(current_batch, f)

    for i in tqdm(range(warm_up, len(dataset), batch_size)):
        current_batch = await asyncio.gather(*tasks[i:i + batch_size])
        responses.append(current_batch)
        with open(f"{checkpoint_dir}/{split}_{prefix}_{i}.json", "w") as f:
            json.dump(current_batch, f)
    return responses

# asyncio.run(main())

In [20]:
responses = await prompt_dataset(dataset, "maival", None)

  0%|          | 0/132 [00:00<?, ?it/s]

["Während der Vorbereitungen für die ProSieben-Show 'TV total Turmspringen' ereignete sich am 14. November 2012 ein Unfall in der Berliner Schwimm- und Sprunghalle im Europasportpark. Ein Teilnehmer verlor beim Sprung vom 3-Meter-Brett die Kontrolle, prallte mit der Stirn gegen die Wasseroberfläche und zog sich möglicherweise eine Rückenmarksverletzung zu. Nachdem er drei Tage intensivmedizinisch behandelt wurde, reichte er später eine Klage gegen die Produktionsfirma Brainpool beim Arbeitsgericht Köln ein und forderte Schmerzensgeld sowie Schadensersatz.", 'Ein Mann erlitt einen Unfall beim Springen im Schwimmbad. Er sprang von einem 3-Meter-Brett und verletzte sich beim Aufprall, da er seinen Kopf nach hinten bog. Im Krankenhaus wurde er gut versorgt, da die Ärzte befürchteten, dass auch sein Rücken verletzt sein könnte. Später verklagte der Mann die Firma, die die Show verantworrtet, weil er Schmerzen hatte und Geld wollte.', 'Oberst Streight hatte die Absicht, die strategischen Ver

  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
responses = await prompt_dataset(test_set, "test", default_experiment)
test_set.add_column("gt", responses)
test_set.to_parquet(f"{output_path}test.parquet")

responses = await prompt_dataset(val_set, "val", default_experiment)
val_set.add_column("gt", responses)
val_set.to_parquet(f"{output_path}val.parquet")


responses = await prompt_dataset(train_set, "train", default_experiment)
train_set.add_column("gt", responses)
train_set.to_parquet(f"{output_path}train.parquet")

  0%|          | 0/34189 [00:00<?, ?it/s]

['freudiger, lautstarker Ausdruck der Begrüßung', 'Kampf, Streit', 'ein lauter Schrei', 'eine regionale Süßspeise', 'malerisch, künstlerisch', 'am wenigsten pittoresken', 'sehr hübsch und reizvoll', 'senkrecht, lotrecht', 'Farbe', 'eine Farbe, die dunkler als Rot', 'eine Farbe', 'eine Farbe', 'Gelb ist hier ein Synonym für Leber oder Fett,', 'das Ergebnis einer Backreaktion', 'Kaffee', 'Gelb ist eine Farbe.', 'Eigelb', 'Name eines berühmten Kaninchenzüchters', 'Brautigamer', 'eine Farbe', 'Wörter sind einzelne Wörter in einem Satz oder Text,', 'kleinste Einheiten der Sprache', 'eine Einheit aus Lauten mit einer bestimmten Bedeutung', 'eine sprachliche Einheit', 'Wörter sind die kleinsten bedeutungstragenden Einheiten einer Sprache.', 'Einzelne Einheiten einer Sprache', 'Eine Gruppe von Bits, die als Einheit verarbeitet werden', 'eine Folge von Bits, die als Einheit verarbeitet wird', 'eine Folge von Buchstaben, die einen bestimmten Sinn oder Begriff ausdrücken', 'Eine Folge von Bits, d

  0%|          | 0/67 [00:00<?, ?it/s]

/home/jfeil/.pyenv/versions/3.10.12/lib/python3.10/typing.py:329: RuntimeWarning: coroutine 'async_prompt' was never awaited
  ev_args = tuple(_eval_type(a, globalns, localns, recursive_guard) for a in t.__args__)


CancelledError: 

In [18]:
default_experiment = Experiment(system_prompt=[
    "Du bist ein Definitionsgenerator. "
    "Du antwortest in Deutsch. "
    "Du bekommst einen Beispielsatz und ein Wort aus dem Satz. "
    "Du antwortest mit der Definition des Wortes im Kontext des Satzes. "
    "Du antwortest nur mit der Definition. "
    "Du zeigst den Bezug auf den Kontext. "
    "Du antwortest in maximal 10 Worten. "
    # "Du antwortest so detailiert wie nötig. "
],
    question_prompt="%s: Was ist in diesem Kontext die Definition von %s?",
    example_prompts=[
        ("Die Liebe überwindet alle Grenzen",
         "Liebe",
         "inniges Gefühl der Zuneigung für jemanden oder für etwas"),
    ])

data = {
    'context_word': "Pferdewurst",
    'context_sentence': "\"Dort liegt zwischen den Steinproben eine dicke rote Wurst - Pferdewurst, ein Geschenk Watzeks, der sie morgens für mich hinterlassen hat.\"",
}


await async_prompt(data, default_experiment)

'typischer deutscher Wurstsort, aus Pferdemuskel hergestellt'

In [ ]:
"Dort liegt zwischen den Steinproben eine dicke rote Wurst - Pferdewurst, ein Geschenk Watzeks, der sie morgens für mich hinterlassen hat.": Was ist die Definition von Pferdewurst?

In [32]:
Du bist ein Definitionsgenerator.  
Du antwortest in Deutsch. 
Du bekommst einen Beispielsatz und ein Wort aus dem Satz.
Du antwortest mit der Definition des Wortes im Kontext des Satzes.
Du antwortest nur mit der Definition. 
Du zeigst den Bezug auf den Kontext.
Du antwortest in maximal 10 Worten. 


{'title': ['Hallo',
  'Hallo',
  'Hallo',
  'Litschi',
  'pittoresk',
  'pittoresk',
  'pittoresk',
  'pittoresk',
  'Gelb',
  'Gelb',
  'Gelb',
  'Gelb',
  'Gelb',
  'Gelb',
  'Gelb',
  'Gelb',
  'Gelb',
  'Gelb',
  'Gelb',
  'Gelb',
  'Wort',
  'Wort',
  'Wort',
  'Wort',
  'Wort',
  'Wort',
  'Wort',
  'Wort',
  'Wort',
  'Wort',
  'Wort',
  'Wort'],
 'context_word': ['Hallo',
  'Hallos',
  'Hallo',
  'Litschi',
  'pittoresken',
  'pittoreskesten',
  'pittoresk',
  'pittoresker',
  'Gelb',
  'Gelb',
  'Gelb',
  'Gelb',
  'Gelb',
  'Gelb',
  'Gelb',
  'Gelb',
  'Gelb',
  'Gelb',
  'Gelb',
  'Gelb',
  'Wörter',
  'Wörtern',
  'Wort',
  'Wort',
  'Wörter',
  'Wörter',
  'Wort',
  'Wort',
  'Wort',
  'Wort',
  'Wort',
  'Wort'],
 'context_sentence': ['Als er die Treppe hinaufkam, wurde er mit großem Hallo empfangen',
  '"Da gab es keine Möglichkeit, mich von vielen Leuten (...) zu verabschieden. Deswegen wird es jetzt ein Tag des großen Hallos und des offiziellen Abschieds werden."',
  